In [1]:
from pathlib import Path
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
!pip install mglearn

In [3]:
data = pd.read_csv('/content/drive/My Drive/NLP/Data/train.csv',encoding='latin1',index_col=0)
data.head()

,text,Target
Id,,
1,@USAirways ! THE WORST in customer service. @...,-1
2,@united call wait times are over 20 minutes an...,-1
3,@JetBlue what's up with the random delay on fl...,-1
4,@AmericanAir Good morning! Wondering why my p...,0
5,@united UA 746. Pacific Rim and Date Night cut...,-1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7320 entries, 1 to 7320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7320 non-null   object
 1   Target  7320 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 171.6+ KB


In [5]:
X_train = data.loc[:, data.columns != 'Target']
X_train.head()

,text
Id,
1,@USAirways ! THE WORST in customer service. @...
2,@united call wait times are over 20 minutes an...
3,@JetBlue what's up with the random delay on fl...
4,@AmericanAir Good morning! Wondering why my p...
5,@united UA 746. Pacific Rim and Date Night cut...


In [6]:
y_train = data.loc[:, data.columns == 'Target']
y_train.head()

,Target
Id,
1,-1
2,-1
3,-1
4,0
5,-1


In [7]:
X = X_train.values.tolist()
y = y_train.values.tolist()

print("type of text_train: {}".format(type(X)))
print("length of text_train: {}".format(len(X)))
print("text_train[6]:\n{}".format(X[6]))

type of text_train: <class 'list'>
length of text_train: 7320
text_train[6]:
['@united See? We were told repeatedly that the pilot was Late Flight and kept getting Late Flightr.  After we boarded, there was a defibrillator issue.']


In [8]:
np.unique(y)

array([-1,  0,  1])

In [9]:
data1 = pd.read_csv('/content/drive/My Drive/NLP/Data/test.csv',encoding='latin1',index_col=0)
X_test = data1.loc[:, data1.columns != 'Target']
y_test = data1.loc[:, data1.columns == 'Target']
X1 = X_test.values.tolist()
y1 = y_test.values.tolist()
print("Number of documents in test data: {}".format(len(X1)))

Number of documents in test data: 7320


In [10]:
X_test.head()

,text
id,
7322,@AmericanAir In car gng to DFW. Pulled over 1h...
7323,"@AmericanAir after all, the plane didnÂÃÂªt ..."
7324,@SouthwestAir can't believe how many paying cu...
7325,@USAirways I can legitimately say that I would...
7326,@AmericanAir still no response from AA. great ...


## **Pre-processing**

### **Tokenization, Stopword removal and Stemming**

In [11]:
import nltk
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# Shorter version of http://sentiment.christopherpotts.net/code-data/happyfuntokenizing.py
# Shorter version adapted from https://github.com/dbamman/anlp19/blob/master/1.words/ExploreTokenization.ipynb

# The order here is important (match from first to last)
import re

# Keep usernames together (any token starting with @, followed by A-Z, a-z, 0-9)
regexes=(r"(?:@[\w_]+)",

# Keep hashtags together (any token starting with #, followed by A-Z, a-z, 0-9, _, or -)
r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",

# Keep words with apostrophes, hyphens and underscores together
#r"(?:[a-z][a-z’'\-_]+[a-z])",

# Keep all other sequences of A-Z, a-z, 0-9, _ together
r"(?:[\w_]+)",

# Everything else that's not whitespace
r"(?:\S)"
)

big_regex="|".join(regexes)

my_extensible_tokenizer = re.compile(big_regex, re.VERBOSE | re.I | re.UNICODE)

def my_extensible_tokenize(text):
    return my_extensible_tokenizer.findall(text)

### **For train data**

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import TweetTokenizer
import spacy
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer

#load tokenizer, stemmer and stop words
stemmer = PorterStemmer() 
stop = set(stopwords.words('english'))

tokens = [my_extensible_tokenize(str(i)) for i in X] #tokenize words in lists
cleaned_list = []
for m in tokens:
    stopped = [i for i in m if str(i).lower() not in stop] #remove stop words
    stemmed = [stemmer.stem(i) for i in stopped] #stem words
    cleaned_list.append(stemmed) #append stemmed words to list

backtodf = pd.DataFrame(cleaned_list) #convert list back to pandas dataframe
remove_NaN = backtodf.replace(np.nan, '', regex=True) #remove None (which return as words (str))
mergeddf = remove_NaN.astype(str).apply(lambda x: ' '.join(x), axis=1) #convert cells to strings, merge columns
mergeddf.head()

# https://stackoverflow.com/questions/47924112/python-read-and-edit-individual-excel-file-rows

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [ ' @usairway ! worst custom servic . @usairwa...
1    [ ' @unit call wait time 20 minut airport wait...
2    [ " @jetblu ' random delay flight 1729 ? chanc...
3    [ ' @americanair good morn ! wonder pre - tsa ...
4    [ ' @unit UA 746 . pacif rim date night cut . ...
dtype: object

### **For test data**

In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import TweetTokenizer

#load tokenizer, stemmer and stop words
stemmer1 = PorterStemmer() 
stop1 = set(stopwords.words('english'))

tokens1 = [my_extensible_tokenize(str(i)) for i in X1] #tokenize words in lists
clean_list = []
for n in tokens1:
    stopped1 = [j for j in n if str(j).lower() not in stop1] #remove stop words
    stemmed1 = [stemmer1.stem(i) for i in stopped1] #stem words
    clean_list.append(stemmed1) #append stemmed words to list

backtodf1 = pd.DataFrame(clean_list) #convert list back to pandas dataframe
remove_NaN1 = backtodf1.replace(np.nan, '', regex=True) #remove None (which return as words (str))
mergeddf1 = remove_NaN1.astype(str).apply(lambda x: ' '.join(x), axis=1) #convert cells to strings, merge columns
mergeddf1.head()

# https://stackoverflow.com/questions/47924112/python-read-and-edit-individual-excel-file-rows

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [ " @americanair car gng dfw . pull 1hr ago - ...
1    [ ' @americanair , plane didnâ \ x89ã \ x9bâªt...
2    [ " @southwestair ' believ mani pay custom lef...
3    [ ' @usairway legitim say would rather driven ...
4    [ ' @americanair still respons AA . great job ...
dtype: object

## **Count Vectorization (Bag of words)**

### **For train data**

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
Xt = vect.fit_transform(mergeddf)
print("X_train:\n{}".format(repr(Xt)))

X_train:
<7320x8340 sparse matrix of type '<class 'numpy.int64'>'
	with 73984 stored elements in Compressed Sparse Row format>


### **For test data**

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vect1 = CountVectorizer().fit(mergeddf1)
Xt1 = vect.transform(mergeddf1)
print("X_test:\n{}".format(repr(Xt1)))

X_test:
<7320x8340 sparse matrix of type '<class 'numpy.int64'>'
	with 69364 stored elements in Compressed Sparse Row format>


## **Cross-validation** **( GridsearchCv on Logistic Regression)**

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

yc=np.ravel(y_train, order = 'C')
param_grid = {'C': [0.45], 'class_weight':[{-1:0.8 ,0:1.1, 1:1.1}]}
grid = GridSearchCV(LogisticRegression(random_state=0,max_iter=1000,multi_class='multinomial'), 
                    param_grid, cv=5, scoring= 'f1_macro')
grid.fit(Xt, yc)
print("Best macro F1_score: {:.4f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

Best macro F1_score: 0.7192
Best parameters:  {'C': 0.45, 'class_weight': {-1: 0.8, 0: 1.1, 1: 1.1}}


In [18]:
from sklearn.metrics import f1_score

print('Train score: {:.4f}'.format(grid.score(Xt, yc)))

Train score: 0.8920


### **Prediction**

In [19]:
test = LogisticRegression(random_state = 0,C=0.45,max_iter=1000,multi_class='multinomial',class_weight={-1:0.8 ,0:1.1, 1:1.1})
test.fit(Xt, yc)

yhat = test.predict(Xt1)
yh = pd.DataFrame(yhat)
yh

,0
0,-1
1,-1
2,-1
3,0
4,1
...,...
7315,0
7316,0
7317,-1
7318,-1


### **Results**

In [20]:
yh.to_csv('/content/drive/My Drive/NLP/Data/result.csv',index = False, header = True)